In [1]:
pip install snowflake-snowpark-python


[notice] A new release of pip available: 22.2.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from snowflake.snowpark.session import Session
import snowflake.snowpark.functions as F

In [7]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [8]:
def snowpark_session_create():
        connection_params = {
            "account": os.getenv('account_snow'),
            "user": os.getenv('user_snow'),
            "password":os.getenv('password_snow')
            #"role":"insert-role-here",
            #"warehouse":"insert-warehouse-here"
   
        }
        session = Session.builder.configs(connection_params).create()
        return session
        
demo_session = snowpark_session_create()

In [9]:
try:
   df = demo_session.sql("select * from  SNOWFLAKE_SAMPLE_DATA.TPCH_SF100.CUSTOMER LIMIT 10000000")
   
   #note the the group_by has a different spelling
   df2 = df.group_by("C_MKTSEGMENT").agg(F.sum("C_ACCTBAL").alias("SUBTOTAL"))
   
   #create a total sum to be used for percentage calculation
   acct_bal_total = df2.agg(F.sum("SUBTOTAL").alias("SUBTOTAL"))
   
   #converting total sum to a pandas df, so as to extract a variable
   pdf  = acct_bal_total.toPandas()
   total_sum = pdf.iloc[0,0]
    
   #create a new column called percentage using the total sum variable
   df2 = df2.with_column("percentage",(df2["SUBTOTAL"]*100)/total_sum)
   df2 = df2.sort(F.col("percentage").desc())
    
   #display the contents of the dataframe 
   df2.show()

finally:
   #closing the connection
   demo_session.close()

-------------------------------------------------
|"C_MKTSEGMENT"  |"SUBTOTAL"     |"PERCENTAGE"  |
-------------------------------------------------
|AUTOMOBILE      |9001308004.66  |20.01024871   |
|FURNITURE       |8998344159.80  |20.00365997   |
|HOUSEHOLD       |8995973688.50  |19.99839033   |
|BUILDING        |8994509561.51  |19.99513552   |
|MACHINERY       |8990277987.42  |19.98572857   |
-------------------------------------------------

